In [15]:
import numpy as np
import plotly.graph_objects as go

In [62]:
evals_history = np.load('evals_history.npy')[:, :]
evals_history.shape

(1080, 100)

In [63]:
risks_history = np.load('risks_history.npy')[:, :]
risks_history.shape

(1080, 100)

In [64]:
def ser(risks):
    prod_safety = 1
    for i in range(len(risks)):
        prod_safety = prod_safety * (1 - (risks[i]/10))    

    ser=(1-prod_safety)/prod_safety

    return ser

In [65]:
def rbf1(risks):
    prod_risk = 0
    for i in range(len(risks)):
        prod_risk = prod_risk + (np.power(0.99, i) * (risks[i]/10))

    if prod_risk < 0.01:
        prod_risk = 0.01

    slc = (1 - prod_risk) / ((prod_risk))

    # overriting rbf_delta
    rbf_delta = 1.0

    rbf = rbf_delta * slc

    return rbf

In [66]:
def rbf(risks):
    prod_risk = 0
    for i in range(len(risks)):
        prod_risk = prod_risk + (np.power(0.99, i) * (risks[i]/10))

    slc = prod_risk

    # overriting rbf_delta
    rbf_delta = 1.0

    rbf = rbf_delta * slc

    return rbf

In [70]:
sers = np.apply_along_axis(ser, 1, risks_history)
rbfs = np.apply_along_axis(rbf, 1, risks_history)


In [71]:
# plot sers and rbfs
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(sers)), y=sers, mode='lines'))
fig.add_trace(go.Scatter(x=np.arange(len(rbfs)), y=rbfs, mode='lines'))
fig.update_layout(title='SER and RBF over time', xaxis_title='Time', yaxis_title='SER/RBF')
fig.show()

In [72]:
#plot all evals
fig = go.Figure()
for i in range(100):
    fig.add_trace(go.Scatter(x=np.arange(len(evals_history[i])), y=evals_history[i], mode='lines'))
fig.update_layout(title='Eval over time', xaxis_title='Time', yaxis_title='Eval')
fig.show()


In [75]:
# get the last positive number from each row of evals_history numpy array
lasts = []
for eval in evals_history:
    # get last positive number from the numpy array
    only_nonzero = eval[eval != 0]
    if len(only_nonzero) == 0:
        last = 0
    else:
        last = only_nonzero[-1]
        
    lasts.append(last)

In [77]:
#plot last positives
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(lasts)), y=lasts, mode='lines'))
fig.update_layout(title='Last eval over time', xaxis_title='Time', yaxis_title='E()')
fig.show()

In [81]:
def reward(evals):
    episode_reward = 0

    non_zero_evals = evals[evals != 0]

    for eval in non_zero_evals:
        episode_reward += eval

    episode_reward = episode_reward / len(non_zero_evals)

    return episode_reward

In [82]:
rewards = np.apply_along_axis(reward, 1, evals_history)

In [83]:
# plot rewards sum
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(rewards)), y=rewards, mode='lines'))
fig.update_layout(title='Rewards avg over time', xaxis_title='Time', yaxis_title='Rewards')
fig.show()